In [1]:
import torch
import torchvision.transforms as transforms
from PIL import Image
from compressai.zoo import models

# Doesnt work
def compress_image_with_pretrained(image_path, quality=1):
    """
    Compress an image using a pre-trained CompressAI model
    
    Args:
        image_path: Path to the input image
        quality: Quality parameter (1-8), higher means better quality but larger size
    
    Returns:
        Compressed and reconstructed image as a PIL Image
    """
    # Load pre-trained model (e.g., Cheng2020Attention)
    model = models['cheng2020-anchor'](quality=quality, pretrained=True)
    model.eval()
    
    # Move to GPU if available
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)
    
    # Load and preprocess image
    img = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.ToTensor()
    ])
    
    x = transform(img).unsqueeze(0).to(device)
    
    # Compress and decompress
    with torch.no_grad():
        compressed = model.compress(x)
        decompressed = model.decompress(compressed['strings'], compressed['shape'])
    
    # Post-process the output
    x_hat = decompressed['x_hat']
    
    # Convert to PIL Image
    out_img = transforms.ToPILImage()(x_hat.squeeze().cpu().clamp_(0, 1))
    
    # Calculate compression stats
    original_size = img.size[0] * img.size[1] * 3  # RGB image
    compressed_size = sum(len(s[0]) for s in compressed['strings'])
    compression_ratio = original_size / compressed_size
    
    return out_img, {
        'original_size_bytes': original_size,
        'compressed_size_bytes': compressed_size,
        'compression_ratio': compression_ratio
    }

In [2]:
# doesnt work
for quality in [1, 4, 8]:
        reconstructed_img, stats = compress_image_with_pretrained(
            '0.png',
            quality=quality
        )
        
        print(f"\nQuality level {quality}:")
        print(f"Original size: {stats['original_size_bytes']/1024:.2f} KB")
        print(f"Compressed size: {stats['compressed_size_bytes']/1024:.2f} KB")
        print(f"Compression ratio: {stats['compression_ratio']:.2f}x")
        
        # Save reconstructed image
        reconstructed_img.save(f'reconstructed_q{quality}.png')

Downloading: "https://compressai.s3.amazonaws.com/models/v1/cheng2020-anchor-1-dad2ebff.pth.tar" to /tmp/xdg-cache/torch/hub/checkpoints/cheng2020-anchor-1-dad2ebff.pth.tar
100%|██████████| 49.1M/49.1M [00:00<00:00, 110MB/s] 
/tmp/ipykernel_902/821993812.py:35: UserWarning: Inference on GPU is not recommended for the autoregressive models (the entropy coder is run sequentially on CPU).
  compressed = model.compress(x)


RuntimeError: Calculated padded input size per channel: (5 x 4). Kernel size: (5 x 5). Kernel size can't be greater than actual input size

In [3]:
# this one works

import torch
import torchvision.transforms as transforms
from PIL import Image
from compressai.zoo import models
import os
from datetime import datetime

def compress_image_with_pretrained(image_path, quality=1, output_prefix=""):
    """
    Compress an image using a pre-trained CompressAI model
    """
    img = Image.open(image_path).convert('RGB')
    
    # Load model
    model = models['bmshj2018-factorized'](quality=quality, pretrained=True)
    model.eval()
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)
    
    # Transform and pad
    x = transforms.ToTensor()(img).unsqueeze(0).to(device)
    _, _, h, w = x.shape
    pad_h = (64 - (h % 64)) % 64
    pad_w = (64 - (w % 64)) % 64
    
    if pad_h > 0 or pad_w > 0:
        x = torch.nn.functional.pad(x, (0, pad_w, 0, pad_h))
    
    # Compress and decompress
    with torch.no_grad():
        compressed = model.compress(x)
        decompressed = model.decompress(compressed['strings'], compressed['shape'])
    
    # Post-process
    x_hat = decompressed['x_hat']
    if pad_h > 0 or pad_w > 0:
        x_hat = x_hat[:, :, :h, :w]
    
    # Convert to PIL Image
    out_img = transforms.ToPILImage()(x_hat.squeeze().cpu().clamp_(0, 1))
    
    # Calculate stats
    original_size = os.path.getsize(image_path)
    compressed_size = sum(len(s[0]) for s in compressed['strings'])
    
    # Save the output
    output_filename = f"{output_prefix}_q{quality}.png"
    out_img.save(output_filename)
    
    return {
        'quality': quality,
        'original_size_bytes': original_size,
        'compressed_size_bytes': compressed_size,
        'compression_ratio': original_size / compressed_size,
        'output_file': output_filename
    }

def compare_quality_levels(image_path, qualities=[1, 4, 6, 8]):
    """
    Compare different quality levels of compression
    """
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_prefix = f"compressed_{timestamp}"
    
    results = []
    print("\nProcessing different quality levels:")
    print("-" * 60)
    print(f"{'Quality':<8} {'Original':<12} {'Compressed':<12} {'Ratio':<8}")
    print("-" * 60)
    
    for quality in qualities:
        result = compress_image_with_pretrained(
            image_path, 
            quality=quality,
            output_prefix=output_prefix
        )
        results.append(result)
        
        print(f"{quality:<8} "
              f"{result['original_size_bytes']/1024:>8.2f} KB "
              f"{result['compressed_size_bytes']/1024:>8.2f} KB "
              f"{result['compression_ratio']:>8.2f}x")
    
    print("-" * 60)
    print(f"\nOutput files saved with prefix: {output_prefix}")
    return results

results = compare_quality_levels('0.png')


Processing different quality levels:
------------------------------------------------------------
Quality  Original     Compressed   Ratio   
------------------------------------------------------------


Downloading: "https://compressai.s3.amazonaws.com/models/v1/bmshj2018-factorized-prior-1-446d5c7f.pth.tar" to /tmp/xdg-cache/torch/hub/checkpoints/bmshj2018-factorized-prior-1-446d5c7f.pth.tar
100%|██████████| 11.5M/11.5M [00:00<00:00, 59.0MB/s]


1         1137.43 KB   105.29 KB    10.80x


Downloading: "https://compressai.s3.amazonaws.com/models/v1/bmshj2018-factorized-prior-4-1ed4405a.pth.tar" to /tmp/xdg-cache/torch/hub/checkpoints/bmshj2018-factorized-prior-4-1ed4405a.pth.tar
100%|██████████| 11.6M/11.6M [00:00<00:00, 57.1MB/s]


4         1137.43 KB   346.11 KB     3.29x


Downloading: "https://compressai.s3.amazonaws.com/models/v1/bmshj2018-factorized-prior-6-9b02ea3a.pth.tar" to /tmp/xdg-cache/torch/hub/checkpoints/bmshj2018-factorized-prior-6-9b02ea3a.pth.tar
100%|██████████| 27.3M/27.3M [00:00<00:00, 47.5MB/s]


6         1137.43 KB   702.19 KB     1.62x


Downloading: "https://compressai.s3.amazonaws.com/models/v1/bmshj2018-factorized-prior-8-5232faa3.pth.tar" to /tmp/xdg-cache/torch/hub/checkpoints/bmshj2018-factorized-prior-8-5232faa3.pth.tar
100%|██████████| 27.9M/27.9M [00:00<00:00, 44.2MB/s]


8         1137.43 KB  1310.84 KB     0.87x
------------------------------------------------------------

Output files saved with prefix: compressed_20241201_170556
